In [ ]:
import pandas as pd
df = pd.read_csv('queryset_CNN_LSTM.csv')
print(df.shape)
print(df.dtypes)

(11, 3)
doc_id         int64
author_id      int64
authorList    object
dtype: object


In [ ]:

for index, row in df.iterrows():
    doc_id = row.doc_id

    author_id = row.author_id

    import ast
    authorList = ast.literal_eval(row.authorList)
    
    candidate = len(authorList)

    test = "ngram-lstm" # change before run

    level = "word"

    iterations = 30

    dropout = 0.5

    samples = 3200

    dimensions = 200

    loc = authorList.index(author_id)

    printstate = (("doc_id = %s, candidate = %s, ") % (str(doc_id), str(candidate)))
    printstate += (("dimensions = %s, samples = %s, ") % (str(dimensions), str(samples)))
    printstate += (("\niterations = %s, dropout = %s, test = %s") % (str(iterations), str(dropout), str(test)))

    print("Current test: %s" % (str(printstate)))
    
    import UpdateDB as db
    case = db.checkCNN(doc_id = doc_id, candidate = candidate, dimensions = dimensions,
                       samples = samples, iterations = iterations, dropout = dropout,
                       test = test)

    if case == False:

        print("Running: %12s" % (str(printstate)))

        import StyloNeuralLSTM as Stylo
        (labels_index, history, train_acc, val_acc, samples) = Stylo.getResults(
            doc_id = doc_id, authorList = authorList[:], 
            level = level, glove = '../../glove/', dimensions = dimensions, 
            samples = samples, nb_epoch = iterations, dropout = dropout, batch_size = 10 )

        (predYList, predY, testY) = Stylo.getTestResults(
            doc_id = doc_id, authorList = authorList[:], labels_index = labels_index,
            level = level, glove = '../../glove/', dimensions = dimensions, 
            samples = samples, nb_epoch = iterations, dropout = dropout, batch_size = 10 )

        loc = testY
        
        test_acc = predY[loc]

        test_bin = 0

        if(predY.tolist().index(max(predY)) == testY):
            test_bin = 1
        
        import UpdateDB as db
        case = db.updateresultCNN(doc_id = doc_id, candidate = candidate, dimensions = dimensions,
                                  samples = samples, iterations = iterations, dropout = dropout,
                                  train_acc = train_acc, val_acc = val_acc,
                                  test_acc = test_acc, test_bin = test_bin,
                                  test = test)
                                     
        del Stylo

        #from keras import backend as K
        #K.clear_session()

        import time
        time.sleep(10)
        
        from IPython.display import clear_output

        clear_output()

    else:
        print("Skipped: %12s" % (str(printstate)))


# import pandas as pd
# df = pd.DataFrame(output)
# df.to_csv("styloout.csv", index = False, encoding='utf-8')

import time
time.sleep(10)

Current test: doc_id = 769, candidate = 3, dimensions = 200, samples = 3200, 
iterations = 30, dropout = 0.5, test = ngram-lstm
Execution completed
Running: doc_id = 769, candidate = 3, dimensions = 200, samples = 3200, 
iterations = 30, dropout = 0.5, test = ngram-lstm


Using Theano backend.
WARNING (theano.gof.compilelock): Overriding existing lock by dead process '28906' (I am process '29087')
Using gpu device 0: GeForce GTX 950 (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5110)
/home/ninadt/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


Level = Word
File used: glove.6B.200d.txt
Found 400000 word vectors.
Execution completed
Read completed
Number of rows: 117
author_id       int64
doc_content    object
dtype: object
Data Frame created: Shape: (13586, 2)
Author:    21  Size:  7849
Author:    59  Size:  4136
Author:   111  Size:  1601
Min: 1601
Max: 7849
Authors [21, 59, 111].
Found 4803 texts.
Found 4803 labels.
Found 68015 unique tokens.
Shape of data tensor: (4803, 1000)
Shape of label tensor: (4803, 3)
Done compiling.
Train on 3842 samples, validate on 961 samples
Epoch 1/30
3842/3842 [==============================] - 124s - loss: 1.1143 - acc: 0.3693 - val_loss: 1.0973 - val_acc: 0.3476
Epoch 2/30
3842/3842 [==============================] - 124s - loss: 1.0236 - acc: 0.4584 - val_loss: 0.9932 - val_acc: 0.4589
Epoch 3/30
3842/3842 [==============================] - 124s - loss: 1.0079 - acc: 0.4742 - val_loss: 1.0802 - val_acc: 0.3673
Epoch 4/30
3842/3842 [==============================] - 124s - loss: 1.0647 - ac

In [ ]:
%tb